In [14]:
from os import listdir
import xmltodict
import pandas
from StringIO import StringIO
%matplotlib inline
from tqdm import tqdm
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from praelexis_file_system_provider.data_access_layer import AzureBlobStorageDataAccessLayer
import PyPDF2
import subprocess
import shutil
import os
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract
import cv2

## Functions used to deskew the images

In [15]:
def orientationScore(image,skipStep = 10):
    totValue = 0
    for row in range(len(image)):
        rowVal = 0
        for col in range(0,len(image[0]),skipStep):
            rowVal += image[row][col]
        totValue += rowVal*rowVal
    return totValue

def getRowCol(image,rowThresh,colThresh):
    #print "rowTh: ",rowThresh, ". ColTh: ",colThresh
    rowArr = np.zeros(len(image))
    colArr = np.zeros(len(image[0]))
    total = 0
    for row in range(len(image)):
        for col in range(len(image[0])):
            rowArr[row] += image[row][col]
            colArr[col] += image[row][col]
            total += image[row][col]
    
    rowLines = []
    colLines = []
    
    for row in range(len(image)):
        if rowArr[row]/len(image)/total<rowThresh:
            rowLines.append(row)
            print 'Row: ', rowArr[row]/len(image)/total
            
    
    for col in range(len(image[0])):
        if colArr[col]/len(image[0])/total<colThresh:
            colLines.append(col)
            print 'Col', colArr[col]/len(image[0])/total
    
    return rowLines,colLines

def findOrient(image,angleRange = 5,skipStep = 10,rLineThresh = 1000,cLineThresh = 1000):
    maxValue = -1
    maxRot  = 0
    imageRows, imageCols = image.shape
    accuracy = 0.1
    
    maxLen = angleRange*2*int(1/accuracy)
    #print maxLen
    for i in range(-1, maxLen):
        #rot = i
        rot = (round(i / 2 - 0.4) + 1) * ((-1) * (i % 2) + ((i + 1) % 2)) * 0.1
        #print rot
        # print("Rotation: ",rot,". i: ",i)
        M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), rot, 1)
        dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))

        value = orientationScore(255-dst,skipStep = 10)
        if value>maxValue:
            maxValue = value
            maxRot = rot
        if int(maxLen/10)!=0 and i%int(maxLen/10)==0:
            print "Percentage complete: ", round(i*100.0/maxLen),"%. Current angle (degree): ",rot
        #print "Current angle: ",rot,". Current value: ", value,". Max value: ",maxValue,". Max rot: ",maxRot
        #cv2.imshow("Rotated Image: ",dst)
        #cv2.waitKey(0)

    M = cv2.getRotationMatrix2D((imageCols / 2, imageRows / 2), maxRot, 1)
    dst = cv2.warpAffine(image, M, (imageCols, imageRows), borderValue=(255, 255, 255))
    
    row,col = getRowCol(dst,rLineThresh,cLineThresh)
    
    print "The best angle was found at: ",maxRot, " degrees."
    #cv2.imshow("Rotated Image: ", dst)
    #cv2.waitKey(0)
    return dst,row,col

In [16]:
#blob_dal = AzureBlobStorageDataAccessLayer('devcapitecdata', 'Gt5z0EAAAm5ZETk//WBnRiQUZ9FDB/BmLmbx5ZPRtDwh6vxTDAB/XzLlM+m/epfnDYcZ8JDfMEGg1p9tBW0iFQ==', 'bank-statements', home_directory='drop/Documentum_line_files')

In [17]:
'''blob_dal.connect()
files = blob_dal.list(path='CI')'''

"blob_dal.connect()\nfiles = blob_dal.list(path='CI')"

In [18]:
'''f = files[0]
f_name = f.directory + f.name
print f_name'''

'f = files[0]\nf_name = f.directory + f.name\nprint f_name'

In [19]:
'''stream = StringIO()
blob_dal.get(f_name, stream)
with open ("temp.pdf", 'w') as tmp:
  stream.seek (0)
  shutil.copyfileobj (stream, tmp)'''

'stream = StringIO()\nblob_dal.get(f_name, stream)\nwith open ("temp.pdf", \'w\') as tmp:\n  stream.seek (0)\n  shutil.copyfileobj (stream, tmp)'

In [20]:
#subprocess.call(["convert","-density", "150", "temp.pdf","tmp.png"])

In [21]:
'''im = plt.imread('tmp-1.png')
#plt.figure(fig_size=(10,10))
plt.imshow(im)
plt.show()'''

"im = plt.imread('tmp-1.png')\n#plt.figure(fig_size=(10,10))\nplt.imshow(im)\nplt.show()"

In [22]:
'''plt.imshow(im)
plt.axis([0,250,300,200])
plt.show()'''

'plt.imshow(im)\nplt.axis([0,250,300,200])\nplt.show()'

## Arguments for processing the image

In [23]:
args = {}
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/rotate.png"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/rotatedDistorted.png"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/RealScan.jpg"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/phonePhoto.jpg"
#args["image"] = "/home/dries/devel/Praelexis/OCR/Images/bankRotatedBackround.jpg"
args["image"] = 'tmp_sharp-1.jpg'
args["output"] = 'OCROutput/'

#Smaller means includes less
args["rLine"] = 0.65e-07#0.75 high
args["cLine"] = 1.4e-07#0.35 high 0.3 low

args["preprocess"] = ""
args["rotationSearch"] = 1
con ="-psm 1 --oem 2 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLNMOPQRSTUVWXYZ-+,.()&:_/\*"
lan = "eng"

#print(pytesseract.image_to_string(Image.open('tmp-0.jpg'), lang='eng'))


## Read and deskew image and find the columns

In [24]:
# load the example image and convert it to grayscale
image = cv2.imread(args["image"])

# cv2.imshow("Image", image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#gray = gray[350:750]

cv2.imwrite(args["output"] + 'Original.png',gray)
#quit()
print("Finding orientation..")
gray,rows,cols = findOrient(gray,angleRange = args["rotationSearch"],skipStep = 10,rLineThresh=args["rLine"],cLineThresh=args["cLine"])

#print("Rows: ",rows)
#print("Cols: ", cols)

grayCopy = gray.copy()

for row in rows:
     cv2.line(grayCopy,(0,row),(511,row),(0,0,0),5)

for col in cols:
    cv2.line(grayCopy,(col,0),(col,511),(0,0,0),5)

cv2.imwrite(args["output"] + 'lines.png',grayCopy)
print("Note: Change percentage to 2 for the extra two rotations at the end.")
print("Image corrected.")

Finding orientation..
Percentage complete:  0.0 %. Current angle (degree):  0.1
Percentage complete:  10.0 %. Current angle (degree):  0.2
Percentage complete:  20.0 %. Current angle (degree):  0.3
Percentage complete:  30.0 %. Current angle (degree):  0.4
Percentage complete:  40.0 %. Current angle (degree):  0.5
Percentage complete:  50.0 %. Current angle (degree):  0.6
Percentage complete:  60.0 %. Current angle (degree):  0.7
Percentage complete:  70.0 %. Current angle (degree):  0.8
Percentage complete:  80.0 %. Current angle (degree):  0.9
Percentage complete:  90.0 %. Current angle (degree):  1.0
Row:  6.227490638966925e-08
Row:  6.25340512167734e-08
Row:  5.707676603422703e-08
Row:  5.820025079406185e-08
Row:  6.366130764176923e-08
Row:  6.299372290827107e-08
Row:  6.26055557021175e-08
Row:  5.805928480866923e-08
Row:  6.435230812980864e-08
Row:  6.412742259458363e-08
Row:  5.990048601810058e-08
Row:  5.6039715267665266e-08
Row:  5.075639814067031e-08
Row:  4.662202451334698e-0

## Generate text form the image using Tesseract

In [25]:
print("Sending image through Tesseract..")
# check to see if we should apply thresholding to preprocess the
# image
if args["preprocess"] == "thresh":
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# make a check to see if median blurring should be done to remove
# noise
elif args["preprocess"] == "blur":
    gray = cv2.medianBlur(gray, 1)

'''cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
cv2.imshow("Input", gray)    
cv2.waitKey(0)'''



# write the grayscale image to disk as a temporary file so we can
# apply OCR to it
#filename = "{}.png".format(os.getpid())
#cv2.imwrite(filename, gray)

# load the image as a PIL/Pillow image, apply OCR, and then delete
# the temporary file
#text = pytesseract.image_to_string(gray,boxes=False)

# This might not be the best approach. I think the system also directly searches for words.
# These word phrases are not included in the config settings.

cv2.imwrite(args["output"] + 'preProcessed.png',gray)

text = pytesseract.image_to_string(gray,lang=lan,boxes=False,config=con)

f = open(args["output"] + 'outputText.txt','w')
f.write(text)
f.close()
#print "Text: ",text


Sending image through Tesseract..


## Estimate the characters and boxes for each characters using Tesseract

In [26]:
data = pytesseract.image_to_string(gray,boxes=True,config=con)
lines = data.split('\n')

array = []


yLen = len(gray)
newPage = gray.copy()

for i in range(len(newPage)):
    for j in range(len(newPage[0])):
        newPage[i][j] = 255

for line in lines:
    entry = line.split(' ')
    #print(entry)
    array.append(entry)
    cv2.rectangle(gray, (int(entry[1]), yLen-int(entry[2])), (int(entry[3]), yLen-int(entry[4])), (0,0,0), 1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    #print("Here: ",entry[0], " ", len(entry[0]))
    try:
        char = str(entry[0])
        #print(char)
        
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,char,loc,font, fondSize,(0,0,255))
        #print char," ",fondSize, " ", loc
    except (TypeError, ValueError):
        print "Not added: ",entry
        xSize = 0.5*(int(entry[3])-int(entry[1]))
        ySize = 0.5*(int(entry[4])-int(entry[2]))
        
        size = xSize
        
        if ySize>xSize:
            size = ySize
        
        fondSize = size/15.0
        loc = (int(entry[1])+int(0.5*(int(entry[3])-int(entry[1]))),yLen-int(entry[2])+int(0.5*(int(entry[4])-int(entry[2]))))
        cv2.putText(newPage,'???',loc,font, fondSize,(0,0,255))
        
        
#data = pytesseract.image_to_boxes(gray)
#os.remove(filename)
#print "Text found:\n",text

#print "Data:\n",array
# show the output images

#for entry in data:
    #print "Entry: ", entry
    #cv2.rectangle(gray, (100, 100), (1000, 1000), (0,0,0), 20)
#cv2.namedWindow('Input', cv2.WINDOW_NORMAL)
#cv2.imshow("Input", gray)
cv2.imwrite(args["output"]+'Boxes.png',gray)
cv2.imwrite(args["output"]+'Estimation.png',newPage)
print "Process done. Go to the output folder to view the results."

Process done. Go to the output folder to view the results.


# Results